In [57]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import joblib
import numpy

# 1. Charger les données
df = pd.read_csv("../vessel-total-clean-final.csv", na_values=["", "NA", "\\N", "na", "n"])

df = df.sort_values(by='mmsi')
# 2. Sélection des colonnes utiles
features = ['sog', 'cog', 'heading', 'length', 'width', 'draft']
df = df[features + ['vessel_type']].dropna()

# 3. Encodage de la variable cible si elle est catégorielle
if df['vessel_type'].dtype == object:
    le = LabelEncoder()
    df['vessel_type'] = le.fit_transform(df['vessel_type'])

# 4. Séparer X et y
X = df[features]
y = df['vessel_type']

# 5. Diviser en ensembles d'entraînement et de test stratifiés
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y  # très important pour préserver la distribution
)

# 6. Prétraitement
num_cols = features
preprocessor = ColumnTransformer([
    ('num', StandardScaler(), num_cols)
])

# 7. Pipeline avec RandomForest
pipeline = Pipeline([
    ('preprocess', preprocessor),
    ('clf', RandomForestClassifier(random_state=42))
])

# 8. GridSearchCV pour tuning
param_grid = {
    'clf__n_estimators': [100, 200],
    'clf__max_depth': [5, 10, 20]
}

grid = GridSearchCV(
    pipeline,
    param_grid,
    cv=5,  
    scoring='accuracy',
    n_jobs=-1
)
grid.fit(X_train, y_train)

# 9. Évaluation sur l'ensemble de test
y_pred = grid.predict(X_test)
print("Best params:", grid.best_params_)
print("Classification report :")
print(classification_report(y_test, y_pred))

# 10. Sauvegarder le modèle
joblib.dump(grid.best_estimator_, "model_vessel_type.pkl")


KeyboardInterrupt: 

In [47]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
import joblib

# 1. Charger les données
df = pd.read_csv("../vessel-total-clean-final.csv", na_values=["", "NA", "\\N", "na", "n"])

# 2. Garder uniquement les colonnes utiles
features = ['sog', 'cog', 'heading', 'length', 'width', 'draft']
df = df[['mmsi'] + features + ['vessel_type']].dropna()

# 3. Conserver une seule ligne par mmsi (par exemple la première apparition)
df = df.drop_duplicates(subset='mmsi', keep='first')   
# Supprimer les classes trop rares (<2 échantillons)
counts = df['vessel_type'].value_counts()
df = df[df['vessel_type'].isin(counts[counts >= 2].index)]

print(df)

# 4. Encoder la variable cible si elle est textuelle
if df['vessel_type'].dtype == object:
    le = LabelEncoder()
    df['vessel_type'] = le.fit_transform(df['vessel_type'])

# 5. Séparer X et y
X = df[features]
y = df['vessel_type']

# 6. Split en train/test (stratifié pour garder la répartition des classes)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

# 7. Pipeline sans transformation
pipeline = Pipeline([
    ('clf', RandomForestClassifier(random_state=42))
])

# 8. GridSearch pour hyperparamètres
param_grid = {
    'clf__n_estimators': [100, 200],
    'clf__max_depth': [10, 20, None]
}

grid = GridSearchCV(
    pipeline,
    param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1
)

grid.fit(X_train, y_train)

# 9. Évaluation
y_pred = grid.predict(X_test)
print("Best params:", grid.best_params_)
print("Classification report :")
print(classification_report(y_test, y_pred))

# 10. Sauvegarde du modèle
joblib.dump(grid.best_estimator_, "model_vessel_type_one_line_per_mmsi.pkl")


          mmsi   sog    cog  heading  length  width  draft  vessel_type
0    636017833  13.4  227.6    227.0   183.0   28.0   10.0           80
5    366954420   0.0    0.0    294.0    30.0    9.0    1.9           60
9    367670000   0.0    0.0    340.0    92.0   20.0    5.0           70
11   255805583   0.1   39.9     64.0   160.0   25.0    6.5           80
12   566996000   0.0    0.0    166.0   199.0   32.0   13.0           70
..         ...   ...    ...      ...     ...    ...    ...          ...
154  538004326   0.1  154.7     67.0   189.0   32.0   12.8           70
155  241770000   0.5  283.2    253.0   274.0   48.0   10.3           80
156  309587000  11.5  279.8    278.0   189.0   31.0   12.8           70
158  538007356   0.0    0.0     83.0   129.0   20.0    6.0           80
159  367518920   0.0    0.0    217.0   188.0   32.0   11.0           80

[104 rows x 8 columns]


c:\Users\lucav\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_split.py:805: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


Best params: {'clf__max_depth': 10, 'clf__n_estimators': 200}
Classification report :
              precision    recall  f1-score   support

          60       1.00      1.00      1.00         2
          70       0.64      0.88      0.74         8
          79       0.00      0.00      0.00         1
          80       0.75      0.67      0.71         9
          89       0.00      0.00      0.00         1

    accuracy                           0.71        21
   macro avg       0.48      0.51      0.49        21
weighted avg       0.66      0.71      0.68        21



c:\Users\lucav\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\lucav\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\lucav\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

['model_vessel_type_one_line_per_mmsi.pkl']

In [49]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import joblib

# 1. Charger les données
df = pd.read_csv("../vessel-total-clean-final.csv", na_values=["", "NA", "\\N", "na", "n"])

# 2. Trier les données par MMSI
df = df.sort_values(by='mmsi')

# 3. Sélection des colonnes utiles
features = ['sog', 'cog', 'heading', 'length', 'width', 'draft']
df = df[features + ['vessel_type']].dropna()

# 4. Séparer X et y
X = df[features]
y = df['vessel_type']

# 5. Diviser en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 6. Prétraitement
num_cols = ['sog', 'cog', 'heading', 'length', 'width', 'draft']

preprocessor = ColumnTransformer([
    ('num', StandardScaler(), num_cols)
])

# 7. Pipeline avec régression logistique
pipeline = Pipeline([
    ('preprocess', preprocessor),
    ('clf', LogisticRegression(random_state=42, max_iter=1000))
])

# 8. GridSearchCV pour tuning
param_grid = {
    'clf__C': [0.1, 1, 10, 100],  # Régularisation inverse (1/C contrôle la pénalisation)
    'clf__penalty': ['l2']  # L2 pour la régularisation de Ridge (standard pour la régression logistique)
}

grid = GridSearchCV(pipeline, param_grid, cv=3, scoring='accuracy', n_jobs=-1)
grid.fit(X_train, y_train)

# 9. Évaluation sur l'ensemble de test
print("Best params:", grid.best_params_)
print("Classification report sur l'ensemble de test :")
print(classification_report(y_test, grid.predict(X_test)))

# 10. Sauvegarder le modèle
joblib.dump(grid.best_estimator_, "model_vessel_type_logistic.pkl")


Best params: {'clf__C': 0.1, 'clf__penalty': 'l2'}
Classification report sur l'ensemble de test :
              precision    recall  f1-score   support

          60       0.72      0.92      0.81      6672
          61       0.00      0.00      0.00       389
          70       0.58      0.50      0.54     17492
          71       0.45      0.34      0.39      1255
          74       0.00      0.00      0.00       334
          79       0.00      0.00      0.00      1151
          80       0.54      0.66      0.59     18520
          82       0.00      0.00      0.00       336
          84       0.00      0.00      0.00       277
          89       0.32      0.13      0.19      1432

    accuracy                           0.58     47858
   macro avg       0.26      0.26      0.25     47858
weighted avg       0.54      0.58      0.56     47858



c:\Users\lucav\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\lucav\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\lucav\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

['model_vessel_type_logistic.pkl']

In [50]:
model_random_forest = joblib.load("model_vessel_type.pkl")

In [51]:
model_logistic = joblib.load("model_vessel_type_logistic.pkl")

In [56]:
model_forest_one_mmsi = joblib.load("model_vessel_type_one_line_per_mmsi.pkl")

In [52]:
import pandas as pd

# Exemple de nouvelles données
new_data = pd.DataFrame({
    'sog': [12.0],
    'cog': [180.0],
    'heading': [190.0],
    'length': [150.0],
    'width': [30.0],
    'draft': [8.0]
})

# Utiliser le modèle directement pour faire une prédiction
prediction = model_random_forest.predict(new_data)
print("Prédiction :", prediction)


Prédiction : [70]


In [53]:
# Exemple de nouvelles données
new_data = pd.DataFrame({
    'sog': [12.0],
    'cog': [180.0],
    'heading': [190.0],
    'length': [150.0],
    'width': [30.0],
    'draft': [8.0]
})

# Utiliser le modèle directement pour faire une prédiction
prediction = model_logistic.predict(new_data)
print("Prédiction :", prediction)


Prédiction : [80]
